# Chapter 05: Coding by Composing

### Defintion of Composing concept 
The composition of two functions `f` and `g` returns a new function `h`, such that:  
`h = f(g) := f ∘ g`

---
#### Functional husbandry

In [1]:
compose = lambda f,g: lambda x: f(g(x))

In [2]:
toUpperCase = lambda x: x.upper()
exclaim = lambda x: f'{x}!'
shout = compose(exclaim, toUpperCase)

In [4]:
shout('send in the clowns')

'SEND IN THE CLOWNS!'

#### What does reduce do ?
- In the functional programming paradigm, reduce is a higher-order function that iteratively applies an operation to elements of a sequence, accumulating them into a single result. 
- Starting with an initial value (or the first element if no initial value is provided), it combines elements one at a time using the specified function.

In [7]:
from functools import reduce

def my_reduce(f, acc0):
    def wrap(li, acc=acc0):
        return acc if len(li)==0 else wrap(li=li[1:], acc=f(acc, li[0]))
    return wrap

In [8]:
head = lambda x: x[0]
my_reverse = my_reduce(lambda acc, x: [x] + acc[:], [])
reverse = lambda iter: reduce(lambda acc, x: [x] + acc[:], iter,[])
last = compose(head, reverse)

Let's take a look at an example of the reduce function from the functools module and compare it to my_reduce function.

In [12]:
example = [1,2,3,4]
print(f"output with functool reduce {reverse(example)}")
print(f"output with my_reduce {my_reverse(example)}")

output with functool reduce [4, 3, 2, 1]
output with my_reduce [4, 3, 2, 1]


In [17]:
arg = ['jumpkick', 'roundhouse', 'uppercut']
print(last(arg))

uppercut


Composition is a relationship between functions that satisfies **Associativity**:  
   `compose(f, compose(g, h))` is equivalent to `compose(compose(f, g), h)`.

In [18]:
print("compose(f, compose(g, h)): ", compose(toUpperCase, compose(head, reverse))(arg))
print("compose(compose(f, g), h): ", compose(compose(toUpperCase, head), reverse)(arg))

compose(f, compose(g, h)):  UPPERCUT
compose(compose(f, g), h):  UPPERCUT


There isn't a built-in **variadic** function for composition in python. \
So we need to define one ourselves :

In [23]:
# Inline version
compose = lambda *fs: reduce(lambda f, g: lambda *a, **kw: f(g(*a, **kw)), fs)

# Expanded version
def compose2(*fs):
    def partComp(f, g):
        def applyComp(*a, **kw):
            return f(g(*a, **kw))
        return applyComp
    return reduce(partComp, fs)

In [35]:
print("Inline version:\t ", compose(toUpperCase, head, reverse)(['jumpkick', 'roundhouse', 'uppercut']))
print("Expanded version:", compose2(toUpperCase, head, reverse)(['jumpkick', 'roundhouse', 'uppercut']))

Inline version:	  UPPERCUT
Expanded version: UPPERCUT


---

#### Definition of Pointfree functions:
- Pointfree functions are defined as functions that operate without explicitly referencing the data they act upon.
- This style relies on first-class functions, currying, and composition, which work seamlessly together to enable concise and expressive function definitions.

In [25]:
import re
from functools import partial

first example

In [26]:
snakeCase = lambda word: re.sub(r's+', '_', word.lower()) # not pointfree function
snakeCase2 = compose(partial(re.sub, r's+', '_'), str.lower) # pointfree function

In [32]:
print("Not a pointfree function: ", snakeCase("Hello sinister man!"))
print("A pointfree function:\t  ", snakeCase2("Hello sinister man!"))

Not a pointfree function:  hello _ini_ter man!
A pointfree function:	   hello _ini_ter man!


second example

In [49]:
initials =  lambda name: '. '.join([ compose(toUpperCase, head)(x) for x in name.split(' ')]) # not pointfree function

map = lambda f, xs: [f(x) for x in xs]
split = lambda sep, x: str.split(x, sep)
intercalate = partial(str.join, '. ')

initials2 = compose(intercalate, partial(map, compose(toUpperCase, head)), partial(split, ' ')) #pointfree function

In [50]:
print("Not a pointfree function: ", initials('hunter stockton thompson'))
print("A pointfree function:\t  ", initials2('hunter stockton thompson'))

Not a pointfree function:  H. S. T
A pointfree function:	   H. S. T


---
#### Debugging
One common error is trying to compose a function before currying it.

In [38]:
angry = compose(exclaim, toUpperCase)

# Error - we end up giving angry an array and we partially applied map with who knows what.
latin = compose(map, angry, reverse)

# Right - each function expects 1 argument
latin2 = compose(partial(map, angry), reverse)

In [41]:
try:
    latin(['frog', 'eyes'])
except Exception as e:
    print(f'Error: {e}')

Error: 'list' object has no attribute 'upper'


In [42]:
latin2(['frog', 'eyes'])

['EYES!', 'FROG!']

To debug this type of issue, you can trace the composition using a function like this:

In [59]:
def trace(tag): 
    def f(x):
        print(tag, x)
        return x
    return f

In [56]:
dasherize = compose(
                partial(str.join, '-'), 
                str.lower, 
                partial(split, ' '), 
                partial(re.sub, r's{2,}', ' ')
            )

In [57]:
try:
    dasherize('The world is a vampire')
except Exception as e:
    print(f'Error: {e}')

Error: descriptor 'lower' for 'str' objects doesn't apply to a 'list' object


In [66]:
dasherize = compose(
                partial(str.join, '-'), 
                str.lower, 
                trace('output after split: '),
                partial(split, ' '), 
                partial(re.sub, r's{2,}', ' ')
            )

In [67]:
try:
    dasherize('The world is a vampire')
except Exception as e:
    print(f'Error: {e}')

output after split:  ['The', 'world', 'is', 'a', 'vampire']
Error: descriptor 'lower' for 'str' objects doesn't apply to a 'list' object


In [64]:
dasherize = compose(
                partial(str.join, '-'), 
                partial(map, str.lower),
                partial(split, ' '), 
                partial(re.sub, r's{2,}', ' ')
            )

In [65]:
dasherize('The world is a vampire')

'the-world-is-a-vampire'

---

#### Category Theory